# OpenWeather DataFrame for ETL Project

We get our Auto Insurance Data Add on our weather data via our API call 

#### Import dependencies

In [98]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
from pprint import pprint
import os
import pandas as pd
import csv

In [100]:
# Import Auto Insurance data
auto_data_path = os.path.join("Resources", "Average Auto Insurance Data.csv")
auto_data_df = pd.read_csv(auto_data_path)

In [101]:
auto_data_df

,State,Zip Code,City,Average Auto Insurance Rate
0,Michigan,48226,Detroit,"$6,329"
1,New York,11212,New York,"$5,703"
2,Louisiana,70117,New Orleans,"$4,601"
3,Nevada,89101,Las Vegas,"$3,768"
4,California,91605,North Hollywood,"$3,767"
...,...,...,...,...
220,Louisiana,70117,New Orleans,"$9,303.50"
221,District Of Columbia,20002,Washington,"$4,440.79"
222,New Jersey,7102,Newark,"$8,512.59"
223,Delaware,6340,Groton,"$3,896.10"


In [111]:
auto_data_list = auto_data_df.values.tolist()
zip_list = auto_data_df.Zip_Code.tolist() 
city_list = auto_data_df.City.tolist()


AttributeError: 'DataFrame' object has no attribute 'Zip'

#### Save configuration and build query URL

In [30]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial code query URL
query_url = f"{url}&units={units}&q="
print(query_url)

# https://api.openweathermap.org/data/2.5/onecall?zip={zip code},{country code}&exclude=hourly,daily&appid={api_key}
# query_url = f"{url}
# api.openweathermap.org/data/2.5/weather?zip={zip code},{country code}&appid={API key}
# pro.openweathermap.org/data/2.5/forecast/hourly?zip={zip code},{country code}&appid={API key}
# https://api.openweathermap.org/data/2.5/onecall?exclude={part}&appid={API key},zip={zip code},{country code}

http://api.openweathermap.org/data/2.5/weather?appid=7e98628ecd4395a6ca3f342c5abe7ccc&units=imperial&q=
http://api.openweathermap.org/data/2.5/weather?&units=imperial&q=


#### Loop through list of cities to request information for each

In [63]:
# Loop with a list of zipcodes
zip_test = ["77077", "77407", "77494"]

# set up lists to hold reponse info
lat_list = []
lon_list = []
hum_list = []
temp_list = []
maxTemp_list = []
minTemp_list = []
cloud_list = []
wind_list = []
city_list = []
# Loop through the list of cities and perform a request for data on each
for zips in zip_test:
    weather_url = f"{query_url}{zips},US&appid={api_key}"
#     print(weather_url)
    response = requests.get(weather_url).json()
#     pprint(response)
    lat_list.append(response['coord']['lat'])
    lon_list.append(response['coord']['lon'])
    temp_list.append(response['main']['temp'])
    maxTemp_list.append(response['main']['temp_max'])
    minTemp_list.append(response['main']['temp_min'])
    hum_list.append(response['main']['humidity'])
    cloud_list.append(response['clouds']['all'])
    wind_list.append(response['wind']['speed'])
    city_list.append(response['name'])

# Create llat lon pairs
latlongList = list(zip(lat_list,lon_list))
latlongList

[(29.71, -95.6), (29.63, -95.73), (29.79, -95.82)]

#### Create a DataFrame from cities, lat, and temp

In [48]:
weather_dict = {"Zip_code": zip_test, "City": city_list, "Max Temperature": maxTemp_list, "Min Temperature": minTemp_list, "Humidity": hum_list, "Wind speed": wind_list, "Cloudiness": cloud_list}
weather_df = pd.DataFrame(weather_dict)
weather_df

,Zip_code,City,Max Temperature,Min Temperature,Humidity,Wind speed,Cloudiness
0,77077,Houston,64,60.80,67,11.41,90
1,77407,Houston,64,61.00,67,11.41,90
2,77494,Houston,64,60.01,67,9.17,90


#### Build a 7 day forecast table

In [65]:
for latlons in latlongList:
    print(f"lat = {latlons[0]}")
    print(f"lon = {latlons[1]}")

lat = 29.71
lon = -95.6
lat = 29.63
lon = -95.73
lat = 29.79
lon = -95.82


In [67]:
# Build 7 day URL
# Save config information.
url = "https://api.openweathermap.org/data/2.5/onecall?"
units = "imperial"
exclude = "hourly,minutely"

# Build partial code query URL
sevenday_query_url = f"{url}&units={units}&exclude={exclude}"
print(sevenday_query_url)

https://api.openweathermap.org/data/2.5/onecall?&units=imperial&exclude=hourly,minutely


In [91]:
# # Loop with a list of zipcodes
# zip_test = ["77077", "77407", "77494"]

# # set up lists to hold reponse info
# latlong_list = []
# lon_list = []

# # Loop through the list of cities and perform a request for data on each
# for zips in zip_test:
#     weather_url = f"{query_url}{zips},US&appid={api_key}"
#     response = requests.get(weather_url).json()
#     lat_list.append(response['coord']['lat'])
#     lon_list.append(response['coord']['lon'])

# # Make Latlong tuples
latlongList = list(zip(lat_list, lon_list))
days = [0, 1, 2]
current_temp_list = []
current_humidity_list = []
current_wind_list = []
Forecast_List = []
Description_List = []
Max_Temp_List = []
Humidity_List = []
rain_List = []

# Loop through the list of tupules and perform a request for data on each
for latlon in latlongList:
    for day in days:
        weather_url = f"{url}lat={latlon[0]}&lon={latlon[1]}&exclude={exclude}&units={units}&appid={api_key}"
        response = requests.get(weather_url).json()
#         pprint(response)
        try:
            current_temp_list.append(response['current']['temp'])
            current_humidity_list.append(response['current']['humidity'])
            current_wind_list.append(response['current']['wind'])
            Max_Temp_List.append(response['daily'][day]['temp']['max'])
            Humidity_List.append(response['daily'][day]['humidity'])
            Forecast_List.append(response['daily'][day]['weather'][0]['main'])
            Description_List.append(response['daily'][day]['weather'][0]['description'])
            rain_List.append(response['daily'][day]['rain'])
        except:
            print(f"Oops, looks like I didn't find some values on day {day}. On to the next one")

Oops, looks like I didn't find some values on day 0. On to the next one
Oops, looks like I didn't find some values on day 1. On to the next one
Oops, looks like I didn't find some values on day 0. On to the next one
Oops, looks like I didn't find some values on day 1. On to the next one
Oops, looks like I didn't find some values on day 0. On to the next one


In [94]:
now_list = Description_List
print(len(now_list))
print(now_list)

9
['overcast clouds', 'broken clouds', 'light rain', 'few clouds', 'broken clouds', 'light rain', 'overcast clouds', 'light rain', 'light rain']


In [ ]:
Forcast_dict = {"Zip_code": zip_test, "City": city_list, "Current Temperature": current_temp_list, "Current Humidity": current_humidity_list, "Current Wind speed": current_wind_list, "Day1 Forecast": Forecast_List, "Wind speed": wind_list, "Cloudiness": cloud_list}
Forcast_df = pd.DataFrame(Forcast_dict)
Forcast_df

In [95]:
current_temp_Dict = {}
current_humidity_Dict = {}
current_wind_Dict = {}
Max_Temp_Dict_7day = {}
Humidity_Dict_7day = {}
Forecast_Dict_7day = {}
Description_Dict_7day = {}
Rain_Dict_7day = {}

# Loop through the list of tupules and perform a request for data on each
for latlon in latlongList:
    for day in days:
        weather_url = f"{url}lat={latlon[0]}&lon={latlon[1]}&exclude={exclude}&units={units}&appid={api_key}"
        response = requests.get(weather_url).json()
#         pprint(response)
        try:
            current_temp_Dict[latlon] = response['current']['temp']
            current_humidity_Dict[latlon] = response['current']['humidity']
            current_wind_Dict = response['current']['wind']
            Max_Temp_Dict_7day = response['daily'][day]['temp']['max']
            Humidity_Dict_7day = response['daily'][day]['humidity']
            Forecast_Dict_7day = response['daily'][day]['weather'][0]['main']
            Description_Dict_7day = response['daily'][day]['weather'][0]['description']
        except:
            print(f"Oops, looks like this might be a problem on day {day}. On to the next one")
        try:
            Rain_Dict_7day = response['daily'][day]['rain']
        except:
            print(f"Oops, looks like I didn't find some values for rain on day {day}. On to the next one")

Oops, looks like this might be a problem on day 0. On to the next one
Oops, looks like I didn't find some values for rain on day 0. On to the next one
Oops, looks like this might be a problem on day 1. On to the next one
Oops, looks like I didn't find some values for rain on day 1. On to the next one
Oops, looks like this might be a problem on day 2. On to the next one
Oops, looks like this might be a problem on day 0. On to the next one
Oops, looks like I didn't find some values for rain on day 0. On to the next one
Oops, looks like this might be a problem on day 1. On to the next one
Oops, looks like I didn't find some values for rain on day 1. On to the next one
Oops, looks like this might be a problem on day 2. On to the next one
Oops, looks like this might be a problem on day 0. On to the next one
Oops, looks like I didn't find some values for rain on day 0. On to the next one
Oops, looks like this might be a problem on day 1. On to the next one
Oops, looks like this might be a pr

In [96]:
# now_list = Description_List
# print(len(now_list))
# print(now_list)
current_temp_Dict

{(29.71, -95.6): 67.15, (29.63, -95.73): 67.35, (29.79, -95.82): 67.57}